In [ ]:
import os
import numpy as np;
import pandas as pd
import matplotlib.pyplot as plt
from scipy import misc
from keras.models import Sequential,Model
from keras.layers.convolutional import Conv3D, Conv2D, UpSampling2D,Conv2DTranspose
from keras.layers.convolutional_recurrent import ConvLSTM2D
from keras.layers.normalization import BatchNormalization
from keras.layers import Dense,MaxPooling2D,TimeDistributed,Input,concatenate,Flatten,Reshape,LSTM,Lambda
import tensorflow as tf
import tensorflow.contrib.layers as c_layers
from keras.preprocessing.image import ImageDataGenerator
import seaborn as sns

In [ ]:
path = 'C:/Users/vkakerbeck/Dropbox/PhD/'
#allI = os.listdir(path+'Data/ObstacleTower/train')

In [ ]:
from keras import backend as K
K.clear_session()

def swish(input_activation):
    """Swish activation function. For more info: https://arxiv.org/abs/1710.05941"""
    return tf.multiply(input_activation, tf.nn.sigmoid(input_activation))

inImg = Input(batch_shape=(None,168, 168, 3),name="input_1")
conv = Conv2D(filters=16, kernel_size=[8, 8], strides=[4, 4],activation=tf.nn.elu, name="conv_1")(inImg)
print(conv)
conv = Conv2D(filters=32, kernel_size=[4, 4], strides=[2, 2],activation=tf.nn.elu, name="conv_2")(conv)
print(conv)
flat = Reshape((19*19*32,))(conv)#Flatten()(conv)
print(flat)
dens = Dense(256,activation=swish,kernel_initializer=c_layers.variance_scaling_initializer(1.0), name="dens_1")(flat)
print(dens)
enc = Dense(256,activation=swish,kernel_initializer=c_layers.variance_scaling_initializer(1.0), name="dens_2")(dens)
print(enc)
de_dens = Dense(20*20*32,activation=swish,kernel_initializer=c_layers.variance_scaling_initializer(1.0), name="dens_3")(enc)
print(de_dens)
shaped = Reshape((20, 20, 32))(de_dens)
print(shaped)
de_conv = Conv2DTranspose(filters=16, kernel_size=[4, 4], strides=[2, 2],activation=tf.nn.elu, name="deconv_1")(shaped)
print(de_conv)
#de_conv = Conv2DTranspose(filters=16, kernel_size=[8, 8], strides=[4, 4],activation=tf.nn.elu, name="deconv_2")(de_conv)
#print(de_conv)
prediction = Conv2DTranspose(filters=3, kernel_size=[8, 8], strides=[4, 4],padding='same',activation=tf.nn.elu, name="deconv_3")(de_conv)
print(prediction)
model = Model(inputs=inImg, outputs=prediction)

#model.compile(optimizer='adadelta',loss='binary_crossentropy',metrics=['accuracy','loss'])
#model.compile(optimizer='adadelta',loss='mean_squared_error',metrics=['accuracy','mse'])
optimizer = tf.train.AdamOptimizer(0.0001)
model.compile(optimizer=optimizer,loss='mean_squared_error',metrics=['accuracy','mse'])

In [ ]:
model.summary()

In [ ]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)


train_generator = train_datagen.flow_from_directory(
    directory=path+'Data/ObstacleTower/',
    target_size=(168, 168),
    color_mode="rgb",
    batch_size=256,
    class_mode="input",
    shuffle=True,
    seed=42
)

In [ ]:
info = model.fit_generator(train_generator,epochs=50)

In [ ]:
model.save_weights(path+'autoencoder/aemodelAdam50E.h5')

In [ ]:
model.load_weights('./Results/TowerTraining/Recordings/Standard/3999_16.100/autoencoder/aemodelAdam50E.h5')

In [ ]:
allI = np.append(allI,info.history['loss'])

In [ ]:
plt.plot(allI)

plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')

plt.show()

In [ ]:
i=60
out = model.predict(data[i].reshape((1,168,168,3)))
plt.figure()
plt.subplot(1,2,1)
plt.imshow(data[i])
plt.axis('off')
plt.subplot(1,2,2)
plt.imshow(out[0])
plt.axis('off')
plt.show()

In [ ]:
folder_name = '3999_16.100'
path = './Results/TowerTraining/Recordings/Standard/'+folder_name+'/'

obs = np.load(path+'visobs.npy', mmap_mode='r')

In [ ]:
out = model.predict(obs)

In [ ]:
np.save(path+'AEOutAdam.npy',out)

In [ ]:
intermediate_layer_model = Model(inputs=model.input,
                                 outputs=[model.get_layer('dens_2').output,model.get_layer('deconv_3').output])
intermediate_output = intermediate_layer_model.predict(obs)

In [ ]:
np.save(path+'AEActAdam.npy',intermediate_output[0])

In [ ]:
i=223
plt.figure(figsize=(20,7))
plt.subplot(1,3,1)
plt.imshow(obs[i])
plt.title('Input',fontsize=20)
plt.axis('off')
plt.subplot(1,3,2)
plt.imshow(intermediate_output[0][i].reshape((8,32)))
plt.title('Encoding',fontsize=20)
plt.axis('off')
plt.subplot(1,3,3)
plt.imshow(intermediate_output[1][i])
plt.title('Output',fontsize=20)
plt.axis('off')
#plt.show()
plt.savefig(path+'autoencoder/Example1.png', bbox_inches='tight')

In [ ]:
i=705
plt.figure(figsize=(20,7))
plt.subplot(1,3,1)
plt.imshow(obs[i])
plt.title('Input',fontsize=20)
plt.axis('off')
plt.subplot(1,3,2)
plt.imshow(intermediate_output[0][i].reshape((8,32)))
plt.title('Encoding',fontsize=20)
plt.axis('off')
plt.subplot(1,3,3)
plt.imshow(intermediate_output[1][i])
plt.title('Output',fontsize=20)
plt.axis('off')
plt.show()
#plt.savefig(path+'autoencoder/Example3.png', bbox_inches='tight')